In [1]:
import pandas as pd
import vectorbt as vbt

In [23]:
# load data
start = "2020-01-01 UTC"
end = "2024-01-01 UTC"
prices = vbt.YFData.download(
    ["META", "AAPL", "AMZN", "NFLX", "GOOG"],
    start=start,
    end=end
).get("Close")

In [25]:
display(prices)

symbol,META,AAPL,AMZN,NFLX,GOOG
Date,,,,,
2020-01-02 05:00:00+00:00,208.795929,72.716064,94.900497,329.809998,68.046196
2020-01-03 05:00:00+00:00,207.691162,72.009125,93.748497,325.899994,67.712273
2020-01-06 05:00:00+00:00,211.602707,72.582916,95.143997,335.829987,69.381874
2020-01-07 05:00:00+00:00,212.060577,72.241554,95.343002,330.750000,69.338585
2020-01-08 05:00:00+00:00,214.210434,73.403633,94.598503,339.260010,69.884987
...,...,...,...,...,...
2023-12-22 05:00:00+00:00,351.732330,192.444580,153.419998,486.760010,142.047211
2023-12-26 05:00:00+00:00,353.165527,191.897888,153.410004,491.190002,142.146729
2023-12-27 05:00:00+00:00,356.151459,191.997269,153.339996,491.790009,140.773239


In [27]:
# Build the moving average indicators using VectorBT's built-in MA class
fast_ma = vbt.MA.run(prices, 10, short_name="fast")
slow_ma = vbt.MA.run(prices, 30, short_name="slow")

In [29]:
# find entry positions when the fast-moving average crosses above the slow-moving average
entries = fast_ma.ma_crossed_above(slow_ma)

In [31]:
display(entries)

fast_window                   10                            
slow_window                   30                            
symbol                      META   AAPL   AMZN   NFLX   GOOG
Date                                                        
2020-01-02 05:00:00+00:00  False  False  False  False  False
2020-01-03 05:00:00+00:00  False  False  False  False  False
2020-01-06 05:00:00+00:00  False  False  False  False  False
2020-01-07 05:00:00+00:00  False  False  False  False  False
2020-01-08 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2023-12-22 05:00:00+00:00  False  False  False  False  False
2023-12-26 05:00:00+00:00  False  False  False  False  False
2023-12-27 05:00:00+00:00  False  False  False  False  False
2023-12-28 05:00:00+00:00  False  False  False  False  False
2023-12-29 05:00:00+00:00  False  False  False  False  False

[1006 rows x 5 columns]

In [33]:
# set up exist positions when the slow-moving average crosses above the fast-moving average
exits = fast_ma.ma_crossed_below(slow_ma)

In [35]:
# Run the backtest using the entry and exist signals
pf = vbt.Portfolio.from_signals(prices, entries, exits)

In [37]:
# visualize the average daiy return for each symbol
pf.total_return().groupby("symbol").mean().vbt.barplot()

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'e3e0d5db-e9d8-4c86-8fc5-99410aa33054',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([ 0.88253537,  0.47098572, -0.0752094 ,  1.71263533,  0.10083658])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [41]:
# see the returns for each symbol by simply holding each throughout the analysis period

(
    vbt
    .Portfolio
    .from_holding(
        prices,
        freq='1d'
    )
    .total_return()
    .groupby("symbol")
    .mean()
    .vbt
    .barplot()
)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'd0de8ee4-976f-45a8-b1cd-096d813f1821',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([1.6318939 , 0.60104537, 1.06132941, 0.68729148, 0.47624392])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [43]:
# split the data into 4 panels
mult_prices, _ = prices.vbt.range_split(n=4)

In [49]:
display(mult_prices, _)

split_idx           0                                                 \
symbol           META        AAPL        AMZN        NFLX       GOOG   
0          208.795929   72.716064   94.900497  329.809998  68.046196   
1          207.691162   72.009125   93.748497  325.899994  67.712273   
2          211.602707   72.582916   95.143997  335.829987  69.381874   
3          212.060577   72.241554   95.343002  330.750000  69.338585   
4          214.210434   73.403633   94.598503  339.260010  69.884987   
..                ...         ...         ...         ...        ...   
246        265.837128  128.817886  160.326004  527.330017  85.768761   
247        266.852295  127.919235  159.263504  514.479980  86.210663   
248        266.145630  128.905792  158.634506  513.969971  86.532639   
249        275.700623  133.516190  164.197998  519.119995  88.385864   
250        275.481628  131.738434  166.100006  530.869995  87.521461   

split_idx           1                                                  \
symbol           META        AAPL        AMZN        NFLX        GOOG   
0          271.878662  129.609100  162.846497  540.729980   87.181076   
1          267.678436  126.405228  159.331497  522.859985   86.004646   
2          269.698914  127.968048  160.925507  520.799988   86.635651   
3          262.074829  123.660477  156.919006  500.489990   86.355484   
4          267.479370  127.880180  158.108002  508.890015   88.941238   
..                ...         ...         ...         ...         ...   
246        328.899872  172.619446  171.037003  614.239990  146.256271   
247        333.667419  173.248474  171.068497  614.090027  146.448853   
248        344.556091  177.228806  169.669495  613.119995  147.365997   
249        344.595917  176.206680  170.660995  610.710022  145.757629   
250        341.331329  176.295166  169.201004  610.539978  145.813873   

split_idx           2                                                  \
symbol           META        AAPL        AMZN        NFLX        GOOG   
0          342.744629  175.135437  168.644501  612.090027  145.314224   
1          334.772186  174.516251  166.716995  602.440002  143.997452   
2          336.951965  178.879913  170.404495  597.369995  144.390594   
3          334.951385  176.609650  167.522003  591.150024  143.735718   
4          322.649384  171.911850  164.356995  567.520020  137.004593   
..                ...         ...         ...         ...         ...   
246        119.198227  133.893204   86.769997  297.959991   89.824554   
247        116.570595  130.710251   83.790001  297.750000   87.843941   
248        117.486290  130.344482   85.250000  294.959991   89.386627   
249        116.331726  128.535522   83.040001  284.170013   87.515495   
250        115.077637  124.591385   81.820000  276.880005   86.052414   

split_idx           3                                                  
symbol           META        AAPL        AMZN        NFLX        GOOG  
0          119.775497  128.436661   84.000000  294.880005   88.311722  
1          124.154854  123.632530   85.820000  294.950012   89.277138  
2          126.772522  124.907700   85.139999  309.410004   88.291809  
3          126.344536  123.583099   83.120003  309.700012   86.360947  
4          129.410095  128.130219   86.080002  315.549988   87.744415  
..                ...         ...         ...         ...         ...  
246        351.732330  192.444580  153.419998  486.760010  142.047211  
247        353.165527  191.897888  153.410004  491.190002  142.146729  
248        356.151459  191.997269  153.339996  491.790009  140.773239  
249        356.639160  192.424713  153.380005  490.510010  140.613983  
250        352.299591  191.380966  151.940002  486.880005  140.265625  

[251 rows x 20 columns]

[DatetimeIndex(['2020-01-02 05:00:00+00:00', '2020-01-03 05:00:00+00:00',
                '2020-01-06 05:00:00+00:00', '2020-01-07 05:00:00+00:00',
                '2020-01-08 05:00:00+00:00', '2020-01-09 05:00:00+00:00',
                '2020-01-10 05:00:00+00:00', '2020-01-13 05:00:00+00:00',
                '2020-01-14 05:00:00+00:00', '2020-01-15 05:00:00+00:00',
                ...
                '2020-12-15 05:00:00+00:00', '2020-12-16 05:00:00+00:00',
                '2020-12-17 05:00:00+00:00', '2020-12-18 05:00:00+00:00',
                '2020-12-21 05:00:00+00:00', '2020-12-22 05:00:00+00:00',
                '2020-12-23 05:00:00+00:00', '2020-12-24 05:00:00+00:00',
                '2020-12-28 05:00:00+00:00', '2020-12-29 05:00:00+00:00'],
               dtype='datetime64[ns, UTC]', name='split_0', length=251, freq=None),
 DatetimeIndex(['2020-12-31 05:00:00+00:00', '2021-01-04 05:00:00+00:00',
                '2021-01-05 05:00:00+00:00', '2021-01-06 05:00:00+00:00',
       

In [81]:
# with each different panel, we can run different combinations of our fast and slow moving average windows
fast_ma = vbt.MA.run(mult_prices, [10, 20], short_name="fast")
slow_ma = vbt.MA.run(mult_prices, [30, 30], short_name="slow")

In [63]:
# find the entries and exits
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)
display(entries)
display(exits)

fast_window     10                                                          \
slow_window     30                                                           
split_idx        0                                  1                        
symbol        META   AAPL   AMZN   NFLX   GOOG   META   AAPL   AMZN   NFLX   
0            False  False  False  False  False  False  False  False  False   
1            False  False  False  False  False  False  False  False  False   
2            False  False  False  False  False  False  False  False  False   
3            False  False  False  False  False  False  False  False  False   
4            False  False  False  False  False  False  False  False  False   
..             ...    ...    ...    ...    ...    ...    ...    ...    ...   
246          False  False  False  False  False  False  False  False  False   
247          False  False  False  False  False  False  False  False  False   
248          False  False  False  False  False  False  False  False  False   
249          False  False  False  False  False  False  False  False  False   
250          False  False  False  False  False  False  False  False  False   

fast_window         ...     20                                            \
slow_window         ...     30                                             
split_idx           ...      2                                  3          
symbol        GOOG  ...   META   AAPL   AMZN   NFLX   GOOG   META   AAPL   
0            False  ...  False  False  False  False  False  False  False   
1            False  ...  False  False  False  False  False  False  False   
2            False  ...  False  False  False  False  False  False  False   
3            False  ...  False  False  False  False  False  False  False   
4            False  ...  False  False  False  False  False  False  False   
..             ...  ...    ...    ...    ...    ...    ...    ...    ...   
246          False  ...  False  False  False  False  False  False  False   
247          False  ...  False  False  False  False  False  False  False   
248          False  ...  False  False  False  False  False  False  False   
249          False  ...  False  False  False  False  False   True  False   
250          False  ...  False  False  False  False  False  False  False   

fast_window                       
slow_window                       
split_idx                         
symbol        AMZN   NFLX   GOOG  
0            False  False  False  
1            False  False  False  
2            False  False  False  
3            False  False  False  
4            False  False  False  
..             ...    ...    ...  
246          False  False  False  
247          False  False  False  
248          False  False  False  
249          False  False  False  
250          False  False  False  

[251 rows x 40 columns]

fast_window     10                                                          \
slow_window     30                                                           
split_idx        0                                  1                        
symbol        META   AAPL   AMZN   NFLX   GOOG   META   AAPL   AMZN   NFLX   
0            False  False  False  False  False  False  False  False  False   
1            False  False  False  False  False  False  False  False  False   
2            False  False  False  False  False  False  False  False  False   
3            False  False  False  False  False  False  False  False  False   
4            False  False  False  False  False  False  False  False  False   
..             ...    ...    ...    ...    ...    ...    ...    ...    ...   
246          False  False  False  False  False  False  False  False  False   
247          False  False  False  False  False  False  False  False  False   
248          False  False  False  False  False  False  False  False  False   
249          False  False  False  False  False  False  False  False  False   
250          False  False  False  False  False  False  False  False  False   

fast_window         ...     20                                            \
slow_window         ...     30                                             
split_idx           ...      2                                  3          
symbol        GOOG  ...   META   AAPL   AMZN   NFLX   GOOG   META   AAPL   
0            False  ...  False  False  False  False  False  False  False   
1            False  ...  False  False  False  False  False  False  False   
2            False  ...  False  False  False  False  False  False  False   
3            False  ...  False  False  False  False  False  False  False   
4            False  ...  False  False  False  False  False  False  False   
..             ...  ...    ...    ...    ...    ...    ...    ...    ...   
246          False  ...  False  False  False  False   True   True  False   
247          False  ...  False  False  False  False  False  False  False   
248          False  ...  False  False  False  False  False  False  False   
249          False  ...  False  False  False  False  False  False  False   
250          False  ...  False  False  False  False  False  False  False   

fast_window                       
slow_window                       
split_idx                         
symbol        AMZN   NFLX   GOOG  
0            False  False  False  
1            False  False  False  
2            False  False  False  
3            False  False  False  
4            False  False  False  
..             ...    ...    ...  
246          False  False  False  
247          False  False  False  
248          False  False  False  
249          False   True  False  
250          False  False  False  

[251 rows x 40 columns]

In [69]:
# backtest the different combinations
pf = vbt.Portfolio.from_signals(
    mult_prices,
    entries,
    exits,
    freq="1D"
)

In [71]:
# visualize the results by grouping total returns by split index and symbol, finding the mean, and plotting
(
    pf
    .total_return()
    .groupby(
        ['split_idx', 'symbol']
    )
    .mean()
    .unstack(level=-1)
    .vbt
    .barplot()
)

FigureWidget({
    'data': [{'name': 'AAPL',
              'showlegend': True,
              'type': 'bar',
              'uid': 'b2d812e8-2c75-4130-8b7e-3f6f0c6aade0',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 0.66134784,  0.29438206, -0.0949337 ,  0.15429983])},
             {'name': 'AMZN',
              'showlegend': True,
              'type': 'bar',
              'uid': '251ab89f-3bd2-47f7-b4e5-747a3f10539d',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 0.43324375, -0.03582058, -0.0632354 ,  0.3530694 ])},
             {'name': 'GOOG',
              'showlegend': True,
              'type': 'bar',
              'uid': 'a1b901f8-21e6-4ff0-8163-d663f2e49794',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 0.17504867,  0.24580724, -0.31914441,  0.12564586])},
             {'name': 'META',
              'showlegend': True,
              'type': 'bar',
              'uid': 'a17719

In [75]:
# get trading statistics from the backtest analysis
pf.orders.stats(group_by=True)

Start                                0
End                                250
Period               251 days 00:00:00
Total Records                      308
Total Buy Orders                   165
Total Sell Orders                  143
Min Size                      0.167918
Max Size                      1.591699
Avg Size                      0.641445
Avg Buy Size                  0.642938
Avg Sell Size                 0.639723
Avg Buy Price               218.033859
Avg Sell Price              219.357003
Total Fees                         0.0
Min Fees                           0.0
Max Fees                           0.0
Avg Fees                           0.0
Avg Buy Fees                       0.0
Avg Sell Fees                      0.0
Name: group, dtype: object

In [77]:
# extract the sharpe ratio for each combination of split
pf.sharpe_ratio()

fast_window  slow_window  split_idx  symbol
10           30           0          META      1.233465
                                     AAPL      2.062475
                                     AMZN      1.929362
                                     NFLX      0.157717
                                     GOOG      1.094471
                          1          META      0.963193
                                     AAPL      2.039331
                                     AMZN      0.064464
                                     NFLX      0.354068
                                     GOOG      1.851288
                          2          META     -1.300484
                                     AAPL     -0.477564
                                     AMZN     -0.541624
                                     NFLX     -0.245790
                                     GOOG     -2.348113
                          3          META      0.248417
                                     AAPL      1.124666
    